# Устанавливаем java и ChromHmm

In [ ]:
!curl -O https://raw.githubusercontent.com/deepjavalibrary/d2l-java/master/tools/fix-colab-gpu.sh && bash fix-colab-gpu.sh

In [ ]:
!curl -O https://raw.githubusercontent.com/deepjavalibrary/d2l-java/master/tools/colab_build.sh && bash colab_build.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   520  100   520    0     0   3333      0 --:--:-- --:--:-- --:--:--  3312
Update environment...
Install Java...
Install Jupyter java kernel...


In [ ]:
!java --list-modules | grep "jdk.jshell"

jdk.jshell@11.0.18


In [ ]:
! wget http://compbio.mit.edu/ChromHMM/ChromHMM.zip

In [ ]:
!unzip /content/ChromHMM.zip

Archive:  /content/ChromHMM.zip
   creating: ChromHMM/ANCHORFILES/
   creating: ChromHMM/ANCHORFILES/ce10/
  inflating: ChromHMM/ANCHORFILES/ce10/RefSeqTES.ce10.txt.gz  
  inflating: ChromHMM/ANCHORFILES/ce10/RefSeqTSS.ce10.txt.gz  
   creating: ChromHMM/ANCHORFILES/ce11/
  inflating: ChromHMM/ANCHORFILES/ce11/RefSeqTES.ce11.txt.gz  
  inflating: ChromHMM/ANCHORFILES/ce11/RefSeqTSS.ce11.txt.gz  
   creating: ChromHMM/ANCHORFILES/ce6/
  inflating: ChromHMM/ANCHORFILES/ce6/RefSeqTES.ce6.txt.gz  
  inflating: ChromHMM/ANCHORFILES/ce6/RefSeqTSS.ce6.txt.gz  
   creating: ChromHMM/ANCHORFILES/danRer10/
  inflating: ChromHMM/ANCHORFILES/danRer10/RefSeqTES.danRer10.txt.gz  
  inflating: ChromHMM/ANCHORFILES/danRer10/RefSeqTSS.danRer10.txt.gz  
   creating: ChromHMM/ANCHORFILES/danRer11/
  inflating: ChromHMM/ANCHORFILES/danRer11/RefSeqTES.danRer11.txt.gz  
  inflating: ChromHMM/ANCHORFILES/danRer11/RefSeqTSS.danRer11.txt.gz  
   creating: ChromHMM/ANCHORFILES/danRer7/
  inflating: ChromHMM/ANC

## Скачиваем файлы

Выравнивания 10-ти разных модификаций гистонов

In [ ]:
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneHelas3H2azAlnRep1.bam -O H2afz.bam
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneHelas3H3k27acStdAlnRep1.bam -O H3k27ac.bam
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneHelas3H3k27me3StdAlnRep1.bam -O H3k27me3.bam
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneHelas3H3k36me3StdAlnRep1.bam -O H3k36me3.bam
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneHelas3H3k04me1StdAlnRep1.bam -O H3k04me1.bam
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneHelas3H3k4me2StdAlnRep1.bam -O H3k4me2.bam
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneHelas3H3k4me3StdAlnRep1.bam -O H3k4me3.bam
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneHelas3H3k79me2StdAlnRep1.bam -O H3k79me2.bam
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneHelas3H3k9acStdAlnRep1.bam -O H3k9ac.bam
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneHelas3H3k09me3AlnRep1.bam -O H3k09me3.bam

Файл с контрольным экспериментом для соответствующего типа клеток

In [ ]:
! wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneHelas3ControlStdAlnRep1.bam -O Control.bam

* Вручную создаем текстовый файл cellmarkfiletable.txt, в котором указываем название типа клеток, разных гистоновых меток, а также соответствующие .bam файлы для эксперимента и контроля. Один и тот же контрольный файл .bam может быть использован для всех экспериментов.

* Более подробно о содержании файла cellmarkfiletable.txt можно посмотреть в руководстве пользователя ChromHMM

* Файл cellmarkfiletable.txt добавляем в github репозиторий для отчета

In [ ]:
import os

control = 'Control.bam'

with open(f'cellmarkfiletable.txt', 'a') as the_file:
  for file in os.listdir():
    if file[-3:] == 'bam' and "Control" not in file:
      the_file.write(f'A549\t{file[:-4]}\t{file}\t{control}\n')

Запускаем ChromHMM с опцией BinarizeBam, чтобы конвертировать профили из ChIP-seq экспериментов (bam-файлы) в табличку из 0 и 1, т.е. чтобы сделать разбивку генома на условные интервалы (бины) длиной 200 п.о.
Для 11 меток программа работает около 5 мин


In [ ]:
!java -mx5000M -jar /content/ChromHMM/ChromHMM.jar BinarizeBam -b 200  /content/ChromHMM/CHROMSIZES/hg19.txt /content/ cellmarkfiletable.txt   binarizedData

Writing to file binarizedData/A549_chr1_binary.txt
Writing to file binarizedData/A549_chr2_binary.txt
Writing to file binarizedData/A549_chr3_binary.txt
Writing to file binarizedData/A549_chr4_binary.txt
Writing to file binarizedData/A549_chr5_binary.txt
Writing to file binarizedData/A549_chr6_binary.txt
Writing to file binarizedData/A549_chr7_binary.txt
Writing to file binarizedData/A549_chrX_binary.txt
Writing to file binarizedData/A549_chr8_binary.txt
Writing to file binarizedData/A549_chr9_binary.txt
Writing to file binarizedData/A549_chr10_binary.txt
Writing to file binarizedData/A549_chr11_binary.txt
Writing to file binarizedData/A549_chr12_binary.txt
Writing to file binarizedData/A549_chr13_binary.txt
Writing to file binarizedData/A549_chr14_binary.txt
Writing to file binarizedData/A549_chr15_binary.txt
Writing to file binarizedData/A549_chr16_binary.txt
Writing to file binarizedData/A549_chr17_binary.txt
Writing to file binarizedData/A549_chr18_binary.txt
Writing to file binari

Запускаем ChromHMM с опцией LearnModel (см. руководство пользователя), которая автоматически определит параметры N разных эпигенетических типов с наиболее выраженными наборами гистоновых меток и присвоит каждому геномному интервалу определенный эпигенетический тип. Количество разных эпигенетических типов (или состояний) выбираете сами (например 10 штук).
Не забудьте, что мы работаем с версией референсного генома человека hg19
Для 11 меток программа работает около 10 минут


In [ ]:
!java -mx5000M -jar /content/ChromHMM/ChromHMM.jar LearnModel -b 200 \
  /content/binarizedData/ /content/data 15 hg19


Writing to file /content/data/transitions_15.txt
Writing to file /content/data/emissions_15.txt
Writing to file /content/data/emissions_15.svg
Writing to file /content/data/emissions_15.png
Writing to file /content/data/transitions_15.png
Writing to file /content/data/transitions_15.svg
Writing to file /content/data/model_15.txt
 Iteration  Estimated Log Likelihood     Change    Total Time (secs)
         1              -9427099.837          -                 27.4
         2              -8322934.901 1104164.936                 54.4
         3              -7950831.893 372103.008                 76.4
         4              -7748845.412 201986.481                101.6
         5              -7658835.414  90009.998                122.2
         6              -7611859.657  46975.757                146.1
         7              -7588116.898  23742.759                167.1
         8              -7578439.203   9677.695                188.1
         9              -7573683.191   4756.012

В результате выполнения команды LearnModel будет создан набор файлов (в том числе HTML страничка) в папке выдачи, которая была указана при запуске. Скачиваем эту папку себе на копьютер и открываем HTML файл.